In [4]:
import numpy as np
import matplotlib.pyplot as plt
import time

BLUE = '\033[94m'
PURPLE = '\033[95m'
CYAN = '\033[96m'
GREEN = '\033[92m'
RED = '\033[91m'
YELLOW = '\033[93m'
ORANGE = '\033[33m'
BLACK_BG = '\033[40;37m'
END = '\033[0m'


def print_message(msg, color, clock, verbose):
    return
    if verbose:
        clock = round(clock, 2) if clock > -1 else ''
        print(f'{color}{clock} {msg}{END}')


# ================================
# Evento
# ================================
class Event:
    def __init__(self, type_, timestamp, id):
        self.type_ = type_
        self.timestamp = timestamp
        self.id = id

    def __str__(self):
        return f'{self.type_} {self.id} {self.timestamp:.2f}'

In [5]:
# ================================
# M/M/1
# ================================
CURRENT_ITERATION = 0


def generate_next_arrival(arrival_rate):
    return np.random.exponential(1 / arrival_rate)


def generate_next_departure(service_rate):
    return np.random.exponential(1 / service_rate)


def print_queue(L):
    print(f'[{", ".join([str(event) for event in L])}]')

def mm1_simulation(arrival_rate, service_rate, max_events, max_queue_len, verbose=True):
    global CURRENT_ITERATION
    CURRENT_ITERATION += 1
    if verbose:
        print_message(f'Iteração #{CURRENT_ITERATION}', BLACK_BG, -1, verbose)
        print_message(
            f'Chegada: {arrival_rate} | Partida: {service_rate} | Iterações: {max_events} | Tamanho máximo da fila: {max_queue_len}',
            BLACK_BG, -1, verbose)
    L = []  # Lista de eventos (Fila de prioridades)
    N = 0  # Número de clientes na fila (variavel de estado)
    customer_number = 0

    # Cria um arquivo csv para armazenar os dados
    file_name = f'mm1_{time.time()}.csv'
    stacktrace = open(file_name, 'w')
    stacktrace.write('eType,N,clock\n')

    def schedule_event(type, timestamp):
        nonlocal customer_number  # Permite acesso à variável customer_number declarada fora da função
        event = Event(type, timestamp, customer_number)
        L.append(event)  # Adiciona evento
        L.sort(key=lambda x: x.timestamp) # Ordena fila por prioridade (timestamp)
        customer_number += 1  # Incrementa o número do cliente para o próximo evento

    # Simulação começa com chegada de cliente
    schedule_event('Arrival', generate_next_arrival(arrival_rate))

    # Plano de controle (enquanto a fila não estiver vazia e o número máximo de
    # iterações não for atingido)
    is_busy = True
    it = 0
    while len(L) > 0 and (customer_number < max_events or max_events == -1):
        it += 1
        e = L.pop(0)  # Remove evento e da fila (de prioridades) de eventos
        clock = e.timestamp  # Atualiza relógio

        # Sortear tempo da próxima chegada e agendar evento
        # Se N = 1, sortear tempo da próxima partida e agendar partida
        if e.type_ == 'Arrival':
            if max_queue_len == -1 or N < max_queue_len:
                schedule_event('Arrival', clock + generate_next_arrival(arrival_rate))
                N += 1
                print_message(f'Cliente chega (N = {N})', GREEN, clock, verbose)
                if N == 1:
                    schedule_event('Departure', clock + generate_next_departure(service_rate))
            else:
                print_message(f'Cliente chega e desiste (N = {N})', RED, clock, verbose)

        # Se N > 0, sortear tempo da próxima partida e agendar partida
        if e.type_ == 'Departure':
            if N > 0:
                # Se servidor estava ocioso, servidor volta a atender
                if not is_busy:
                    print_message(f'Servidor volta a atender (N = {N})', CYAN, clock, verbose)
                N -= 1
                is_busy = True
                schedule_event('Departure', clock + generate_next_departure(service_rate))
                print_message(f'Cliente sai (N = {N})', YELLOW, clock, verbose)

        stacktrace.write(f'{e.type_},{N},{clock}\n')

        # Se N = 0 e servidor estava ocupado, servidor fica ocioso
        if N == 0 and is_busy:
            print_message(f'Servidor fica ocioso (N = {N})', BLUE, clock, verbose)
            is_busy = False

        if max_events == -1 and it == 100000:
            break

    stacktrace.close()
    return file_name

def mm1_run_many(arrival_rate, service_rate, max_events, max_queue_len, i=2):
    counter = 0
    file_trace = []
    while counter < i:
        print(f'{PURPLE}Simulação #{counter + 1}{END}')
        file_trace.append(mm1_simulation(arrival_rate, service_rate, max_events, max_queue_len))
        counter += 1
    return file_trace

In [6]:
# ================================
# Ruína do Apostador
# ================================

def gambler_ruin(lambda_, mi_, goal, starting_amount, infinity=False, verbose=True):
    p = lambda_ / (lambda_ + mi_)  # Probabilidade de ganhar
    q = mi_ / (lambda_ + mi_)  # Probabilidade de perder
    N = starting_amount  # Equivalente a primeira chegada
    round = 0  # Número de rodadas ("clock")

    # Cria um arquivo csv para armazenar os dados
    file_name = f'gambler_{time.time()}.csv'
    stacktrace = open(file_name, 'w')
    stacktrace.write('clock,N\n')

    def schedule_event(p, timestamp):
        # Escolhe em um intervalo de 0 a 1 a propabilidade de escolher p ou q
        if np.random.rand() < p:
            return Event('Win', timestamp, 1)
        else:
            return Event('Lose', timestamp, -1)

    # Plano de controle (enquanto o apostador não atingir o objetivo ou falir)
    while (N > 0 and (N < goal or goal == -1)) or infinity:
        round += 1
        event = schedule_event(p, round)
        N += event.id

        if event.type_ == 'Win':
            print_message(f'{round}: Jogador ganhou {event.id} | Montante: {N}', PURPLE, round, verbose)

        if event.type_ == 'Lose':
            print_message(f'{round}: Jogador perdeu {event.id} | Montante: {N}', BLUE, round, verbose)

        # Escreve no arquivo
        stacktrace.write(f'{round},{N}\n')

        # No modo de execucao infinito, sempre que o apostador falir, ele volta a ter 1 real
        if N == 0 and infinity:
            N = 1

        if infinity and round == 100000:
            break

    # Imprime resultado
    if N == goal:
        print_message(f'O apostador atingiu o objetivo de {goal} em {round} rodadas', GREEN, round, verbose)
    else:
        print_message(f'O apostador faliu em {round} rodadas', RED, round, verbose)

    stacktrace.close()
    return file_name

def gambler_run_many(lambda_, mi_, goal, starting_amount, i=2):
    counter = 0
    file_trace = []
    while counter < i:
        print(f'{PURPLE}Simulação #{counter + 1}{END}')
        file_trace.append(gambler_ruin(lambda_, mi_, goal, starting_amount))
        counter += 1
    return file_trace

In [42]:
# Calcula Ro
def calculaRo(lambda_, mu_):
    ro = lambda_ / mu_
    return ro

# Fracao de tempo que o sistema fica ocioso
def fracaoTempoSistemaOcioso(ro):
    pi_zero = 1 - ro
    return pi_zero

# Tamanho medio da fila
def tamanhoMedioFila(ro):
    Nq = (ro**2) / (1 - ro)
    return Nq

# Tempo medio do cliente no sistema
def tempoMedioClienteSistema(lambda_, N):
    W = N / lambda_
    return W

# Numero medio de clientes no sistema
def numeroMedioClientesSistema(ro):
    N = ro / (1 - ro)
    return N

def calculadoraAnalitica(lambda_, mu_):
    ro = calculaRo(lambda_, mu_)
    pi_zero = fracaoTempoSistemaOcioso(ro)
    Lq = tamanhoMedioFila(ro)
    W = tempoMedioClienteSistema(lambda_, N)

    resultados = {
        'lambda': lambda_,
        'mu': mu_,
        'ro': ro,
        'pi_zero': pi_zero,
        'Lq': Lq,
        'L': N,
        'W': W
    }
    print(resultados)
    return ro, pi_zero, N, W


# Temos que calcular:
#   E_N = Média do numero de clientes no sistema
#   W = Tempo médio de espera de cada cliente no sistema
#   CDF(E_N)
#   CDF(W)

def calculadoraSimulacao(lista_arquivos):
    for arquivo in lista_arquivos:
        iterr = 0
        arrivals, services = 0, 0
        apx_lambda, apx_mu = 0, 0
        pi_zero = 0 # Fração de tempo que o sistema fica ocioso
        E_de_N = 0
        file = open(arquivo, 'r')
        Ene, clock , N_acum = 0, 0, 0
        Ene_ant, clock_ant = Ene, clock
        T_med_entre_arr = 0
        T_med_na_fila_p_cli = 0
        T_act_arr, T_arr_ant, T_arr_acum = 0, 0, 0
        T_act_ser, T_ser_ant, T_ser_acum = 0, 0, 0
        T_med_arr_p_cli, T_med_ser_p_cli = 0, 0
        # Drop the first line
        file.readline()
        CDF = {}
        for line in file.readlines():
            iterr += 1
            eType, Ene, clock = line.split(',')
            eType = eType
            Ene = int(Ene)
            clock = float(clock)
            N_acum += Ene
            E_de_N = N_acum / iterr

            if (Ene not in CDF):
                CDF[Ene] = 1
            else:
                CDF[Ene] += 1

            # Aproximate the mean of arrival rate
            if eType == 'Arrival':
                arrivals += 1
                T_act_arr = clock
                T_arr_acum += T_act_arr - T_arr_ant
                T_med_entre_arr = T_arr_acum / arrivals
                T_med_na_fila_p_cli = (E_de_N - 1) * T_med_ser_p_cli
                apx_lambda = arrivals / clock
                T_arr_ant = T_act_arr
            else:
                services += 1
                if (Ene_ant == 0):
                    pi_zero += clock - clock_ant
                T_act_ser = clock
                T_ser_acum += T_act_ser - T_ser_ant
                T_med_ser_p_cli = T_ser_acum / services
                apx_mu = services / clock
                T_ser_ant = T_act_ser

            _cdf = [(CDF[Ene] / iterr) for Ene in CDF]
            _cdf.sort()
            _cdf_sum = 0
            for Ene in _cdf:
                _cdf_sum += Ene
            Dabliu = E_de_N / apx_lambda
            Dabliu2 = T_med_na_fila_p_cli + T_med_ser_p_cli
            
            Ene_ant, clock_ant = Ene, clock
            if (iterr % 100 == 0):
              print(f'sum: {_cdf_sum}, CDF: {_cdf}')
              print(f'iterr: {iterr}, lambda: {apx_lambda}, mu: {apx_mu}, E[N]: {E_de_N}, W: {Dabliu} | {Dabliu2}, T. méd. entre chegadas: {T_med_entre_arr}')

In [43]:
calculadoraSimulacao(mm1_run_many(2, 4, 10000, -1, 1))

Simulação #1
sum: 1.0, CDF: [0.02, 0.07, 0.16, 0.33, 0.42]
iterr: 100, lambda: 1.9613384655680384, mu: 2.9067635806100727, E[N]: 0.94, W: 0.47926455147952207 | 0.3364482805922637, T. méd. entre chegadas: 0.5098559058292789
sum: 1.0, CDF: [0.01, 0.035, 0.1, 0.335, 0.52]
iterr: 200, lambda: 1.972163904319331, mu: 3.30921432945865, E[N]: 0.68, W: 0.34479892797485 | 0.20794970683377711, T. méd. entre chegadas: 0.5070572470218383
sum: 1.0, CDF: [0.0033333333333333335, 0.006666666666666667, 0.01, 0.023333333333333334, 0.043333333333333335, 0.11, 0.32666666666666666, 0.4766666666666667]
iterr: 300, lambda: 1.9451834648210231, mu: 3.1267683114392346, E[N]: 0.8833333333333333, W: 0.45411312059174275 | 0.28296933797653834, T. méd. entre chegadas: 0.5140903251981994
sum: 1.0, CDF: [0.0025, 0.005, 0.0175, 0.0425, 0.0575, 0.1075, 0.3075, 0.46]
iterr: 400, lambda: 1.9925000723536819, mu: 3.146237156685896, E[N]: 1.0, W: 0.5018820394915867 | 0.31458359707422295, T. méd. entre chegadas: 0.501882039491